<h1><center>HW2 Scrape Move Reviews</center></h1>

Choose one of your favourite movie on IMDB website (e.g. for the movie "Top Gun: Maverick", the URL is :https://www.imdb.com/title/tt1745960/reviews?sort=totalVotes&dir=desc&ratingFilter=0

In [1]:
pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 995 kB 5.3 MB/s 
     |████████████████████████████████| 140 kB 56.8 MB/s 
     |████████████████████████████████| 384 kB 54.6 MB/s 
     |████████████████████████████████| 58 kB 2.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


- Q1. Write a function to scrape all **reviews on the first page**, including, 
    - **title** (see (1) in Figure)
    - **reviewer's name** (see (2) in Figure)
    - **date** (see (3) in Figure)
    - **rating** (see (4) in Figure)
    - **review content** (see (5) in Figure. For each review text, need to get the **complete text**.)
    - **helpful** (see (6) in Figure). 
    - If a field, e.g. rating, is missing, use `None` to indicate it. 
- `Function Input`: book page URL
- `Function Output`: save all reviews as a DataFrame of columns (`title, reviewer, date, rating, review, helpful`). E.g., for the given URL, you can get 25 reviews.

    
![alt text](IMDB.png "IMDB")

In [2]:

import requests
from bs4 import BeautifulSoup  
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
def getReviews(page_url):
    
    page = requests.get(page_url)
    if page.status_code==200:
        soup = BeautifulSoup(page.content,'html.parser')
        
        
    div_ = soup.find_all("div", class_="lister-item-content")
    
    titles=[]
    review_names=[]
    dates=[]
    scores=[]
    texts=[]
    helpfuls=[]
    
    for idx,div in enumerate(div_):
        title = div.a.text
        titles.append(title.strip())
         
        
        if div.find("div", class_="ipl-ratings-bar"):
            score = div.find("div", class_="ipl-ratings-bar").find("span", class_="rating-other-user-rating").find_all("span")[0].text
            scores.append(score)
        else:
            scores.append("None")
                       
            
        if div.find("div", class_="display-name-date"):
            review_name = div.find("div", class_="display-name-date").find("a").text
            review_names.append(review_name)
        else:
            review_names.append("None")
            
        if div.find("div", class_="display-name-date"):
            date = div.find("div", class_="display-name-date").find("span", class_="review-date").text
            dates.append(date)
        else:
            dates.append("None")
            
        if div.find("div", class_="content"):
            text = div.find("div", class_="content").find("div", class_="text").text
            texts.append(text)
        else:
            texts.append("None")
            
        if div.find("div", class_="content"):
            helpful = div.find("div", class_="content").find("div", class_="actions").text.split("\n")[1]
            helpfuls.append(helpful)
        else:
            helpfuls.append("None")
            
    output = {'Title': titles,'name':review_names,'Date': dates,'Rating':scores,'Review':texts,'Helpful':helpfuls}
    reviews_tb = pd.DataFrame(output)
    return reviews_tb
            

page_url = 'https://www.imdb.com/title/tt0120363/reviews?sort=curated&dir=desc&ratingFilter=1'
getReviews(page_url)
    




,Title,name,Date,Rating,Review,Helpful
0,Moronically Atrocious,Miura88,28 August 2009,1,How this has been rewarded monumental praise b...,10 out of 46 found this he...
1,"Not as funny, original or overall better than ...",Silverzero,31 August 2002,1,I just noticed that Toy Story 2 has been voted...,6 out of 39 found this hel...
2,"I give a ""2"", like the number of this episode",blanka76,3 October 2001,1,"Unbelievable! How can it be possible? Hey, up ...",5 out of 34 found this hel...
3,Like the Original...Awful!,HonestReviews,29 March 2016,1,"Having seen the awful Toy Story, (be sure to s...",3 out of 23 found this hel...


In [6]:
(getReviews(page_url))

,Title,name,Date,Rating,Review,Helpful
0,Moronically Atrocious,Miura88,28 August 2009,1,How this has been rewarded monumental praise b...,10 out of 46 found this he...
1,"Not as funny, original or overall better than ...",Silverzero,31 August 2002,1,I just noticed that Toy Story 2 has been voted...,6 out of 39 found this hel...
2,"I give a ""2"", like the number of this episode",blanka76,3 October 2001,1,"Unbelievable! How can it be possible? Hey, up ...",5 out of 34 found this hel...
3,Like the Original...Awful!,HonestReviews,29 March 2016,1,"Having seen the awful Toy Story, (be sure to s...",3 out of 23 found this hel...


- Bonus point. Modify the function you defined in Q1 to Scrape **all reviews on the first five page**.


In [7]:
def getReviews2():
    
    base_url = "https://www.imdb.com/title/tt0120363/reviews?sort=curated&dir=desc&ratingFilter="

    df = getReviews(base_url+'1')
    for x in range(2, 6):
        url = base_url + str(x)
        
        df = pd.concat([df, getReviews(url)], ignore_index=True)

    print(df)

getReviews2()

                                                Title             name  \
0                               Moronically Atrocious          Miura88   
1   Not as funny, original or overall better than ...       Silverzero   
2       I give a "2", like the number of this episode         blanka76   
3                          Like the Original...Awful!    HonestReviews   
4                                  Masterpiece sequel        kenandraf   
5                               Almost Unwatchable...         ravenosa   
6         It sucks It doesn't live up to the original       angeltania   
7                           Highly overrated movie!!!             Rept   
8                    Heaven forbid  I didn't like it        FilmOtaku   
9                         doesn't hold up as an adult        FeastMode   
10                   I must be from another planet...       LDB_Movies   
11                              Newman the toy hunter     ThunderKing6   
12               Joan Cusack has an an